# Purpose
- This file is a scratch-pad for sql queries

In [1]:
%load_ext sql
%sql duckdb:///../data/referral.db --alias referralsDB

c:\Users\MatthewLyn\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


There's a new jupysql version available (0.7.0), you're running 0.5.5. To upgrade: pip install jupysql --upgrade


In [2]:
%sql referralsDB

In [3]:
%%sql
CREATE OR REPLACE TABLE coc as

SELECT DISTINCT * FROM read_csv_auto('../data/reference/coc.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
818829


In [4]:
%%sql 
CREATE OR REPLACE TABLE team as 

SELECT DISTINCT * FROM read_csv_auto('../data/reference/user_list.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
266


In [5]:
%sql CREATE OR REPLACE TABLE hcpcs as SELECT DISTINCT * FROM read_csv_auto('../data/reference/cpt_hcpcs.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16991


In [6]:
%sql CREATE OR REPLACE TABLE icd10cm as SELECT DISTINCT * FROM read_csv('../data/reference/icd10cm.csv', header=True, columns={'code_value': 'VARCHAR', 'code_long_description': 'VARCHAR'})

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
95387


In [7]:
%%sql 
CREATE OR REPLACE TABLE cln_referrals as 

SELECT DISTINCT * FROM read_parquet('../data/clean_referrals/*.parquet')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
100358


- No Auth Required ->	Appointment Scheduled or Member to Schedule or Unable to Contact
- Approved	       ->   Appointment Scheduled or Member to Schedule or Unable to Contact
- Denied	       ->   Open
- Withdrawn	       ->   Open

In [8]:
%%sql 
CREATE OR REPLACE TABLE bi_referrals as

WITH main as (

    SELECT DISTINCT rf.*
    , COALESCE(tm.Team, 'N/A') as User_Team

    , UPPER("Last Name") || UPPER("First Name") as patient_name

    , c.health_plan
    , c.care_partner_name
    , c.care_partner_group

    , UPPER("Last Name") || UPPER("First Name") || "DOB" as patient_id

    , patient_id || coalesce("Plan ID", 'NA') || coalesce("Referring Provider NPI", 'NA') || "Referral Date" || coalesce("Procedure", 'NA') || coalesce(Specialty, 'NA') as Referral_keyid

    , CASE
        WHEN "Approval Status" IN ('No HP Auth Required', 'HP Approved', 'Approved (comments required)', 'Complete/no Auth# needed')  
            AND "Visit Status" IN ('Appointment Scheduled', 'Member to Schedule', 'Unable to Contact', 'Denied', 'Withdrawn')
            THEN 'Completed'
        WHEN "Approval Status" IN ('Denied- Per Insurance Plan', 'Denied- Per Medical Director Review')
            AND "Visit Status" = 'Open'
            THEN 'Completed'
        ELSE "Referral Status" END as "Updated Status"

    , RIGHT("Procedure", 5) as Proc_code
    , pc.code_short_description as Proc_name 

    , FORMAT(
        '({}) {}-{}'
        , substring("Mobile Phone", 1, 3), substring("Mobile Phone", 3, 3), substring("Mobile Phone", 6, 4)
    ) as "Fmt Mobile Phone"

    , FORMAT(
        '({}) {}-{}'
        , substring("Home Phone", 1, 3), substring("Home Phone", 3, 3), substring("Home Phone", 6, 4)
    ) as "Fmt Home Phone"

    , icd.code_long_description AS "Diagnosis Description"

    FROM cln_referrals AS rf

    LEFT JOIN icd10cm AS icd
        ON trim(rf."Diagnosis") = trim(icd.code_value)

    LEFT JOIN hcpcs AS pc
        on RIGHT("Procedure", 5) = pc.code_value

    LEFT JOIN team as tm
        on  "User_FName" = tm.Fname
        AND "User_LName" = tm.Lname

    LEFT JOIN coc as c
        ON rf."Plan ID" = c.member_id
        AND date_part('month', "Referral Date":: DATE) = c.month_ym:: INTEGER
        AND date_part('year', "Referral Date":: DATE) = c.year_ym:: INTEGER

    WHERE "Visit Status" IS NOT NULL
)

, max_updatedt as (
    SELECT DISTINCT
        Referral_keyid
        , Update_DT
        , MAX(Update_DT) OVER (PARTITION BY Referral_keyid) as Last_UpdateDT

    FROM main 
)

SELECT DISTINCT m.* 

FROM main as m

INNER JOIN max_updatedt as dt
    ON m.Update_DT = dt.Last_UpdateDT
    AND m.Referral_keyid = dt.Referral_keyid

*  (referralsDB) duckdb:///../data/referral.db


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Done.


Count
96094


In [9]:
%%sql

SELECT DISTINCT * FROM bi_referrals

LIMIT 10

*  (referralsDB) duckdb:///../data/referral.db
Done.


C0,Center,Clinic,Referring Provider,Referring Provider NPI,Referral Date,Last Name,First Name,Sex,DOB,pat Status,Language,Address,Mobile Phone,Home Phone,PCP,Referred to Specialist,Specialist NPI,Referral Status,Network,Specialty,Diagnosis,Procedure,Approval Status,Visit Status,Referral Type,Patient Type,Health Plan,Plan ID,LOB,Auth No,Last Update,Update_DT,file_source,User_Fname,User_LName,User_Team,patient_name,health_plan,care_partner_name,care_partner_group,patient_id,Referral_keyid,Updated Status,Proc_code,Proc_name,Fmt Mobile Phone,Fmt Home Phone,Diagnosis Description
6889,"Centrum Medical Holdings, LLC",CH - W Hialeah (3F),"Izaela Bauza, ARNP",1689030900,2023-03-02,RIVERO,ROBERTO,M,1968-09-25,Active,Spanish,"225 E 41 STREETHIEALEH, FL, 33013",None,7866312427,"Bauza, Izaela",APT PENDING BY PTE,None,Open,Out of network,Ophthalmology,H54.1131,OFFICE OUTPATIENT VISIT 25 MINUTES - 99214,Reviewed- Ready to be processed,Pending,External,Adult,Oscar Insurance Company of Florida,OSC79937710-01,Exchange,n/a,"RIVAS, ARELYS",Mar 2 2023 8:18AM,../data/raw_referrals/excels/FL REFERRALS 3_1-3_30.xlsx,ARELYS,RIVAS,N/A,RIVEROROBERTO,OSCAR IFP,None,None,RIVEROROBERTO1968-09-25,RIVEROROBERTO1968-09-25OSC79937710-0116890309002023-03-02OFFICE OUTPATIENT VISIT 25 MINUTES - 99214Ophthalmology,Open,99214,OFFICE/OUTPATIENT VISIT EST,None,(786) 663-1242,BLINDNESS OD CATEGORY 3 LOW VISION OS CATEGORY 1
7173,"Centrum Medical Holdings, LLC",CH - Coral Springs,"Katherine Perez, MD",1164487427,2023-03-22,AMAYA,SANDRA,F,1980-04-09,Active,Spanish,"9905 Westwood Dr Unit 34Tamarac, FL, 33321",9544807624,9544807624,"Perez, Katherine",None,None,Open,Out of network,Diagnostic Radiology,Z12.31,Screening Mammogram - 77067,Complete/no Auth# needed,Pending,External,Adult,Ambetter (Centrum Contract),U9684690302,Exchange,N/A,"SORIA, JAQUELINE",Mar 22 2023 10:59AM,../data/raw_referrals/excels/FL REFERRALS 3_1-3_28.xlsx,JAQUELINE,SORIA,N/A,AMAYASANDRA,AMBETTER FL IFP,None,None,AMAYASANDRA1980-04-09,AMAYASANDRA1980-04-09U968469030211644874272023-03-22Screening Mammogram - 77067Diagnostic Radiology,Open,77067,SCR MAMMO BI INCL CAD,(954) 448-0762,(954) 448-0762,ENCOUNTER SCREENING MAMMO MALIG NEOPLASM BREAST
8037,"Centrum Medical Holdings, LLC",Home Visit,"Lucia Cordero, APRN",1235716960,2023-03-15,LARA FORNER,ALICIA,F,1989-05-03,Active,Spanish,"19225 NW 53RD CIRCLE PLMIAMI GARDENS, FL, 33055",7866411972,7866411972,"Gomez Padron, Maria",by pt,None,Open,Out of network,Diagnostic Radiology,E03.9,ULTRASOUND OF THYROID - 76536,Complete/no Auth# needed,Completed,External,Adult,Oscar Insurance Company of Florida,OSC79922723-01,Exchange,by pt,"GUTIERREZ, LISETE",Mar 15 2023 4:35PM,../data/raw_referrals/excels/FL REFERRALS 3_1-3_29.xlsx,LISETE,GUTIERREZ,N/A,LARA FORNERALICIA,OSCAR IFP,None,None,LARA FORNERALICIA1989-05-03,LARA FORNERALICIA1989-05-03OSC79922723-0112357169602023-03-15ULTRASOUND OF THYROID - 76536Diagnostic Radiology,Open,76536,US EXAM OF HEAD AND NECK,(786) 664-1197,(786) 664-1197,HYPOTHYROIDISM UNSPECIFIED
2452,"Centrum Medical Holdings, LLC",CH - Margate,"Caridad Valdes, APRN",1609522788,2023-03-01,MARCELLUS,ANTUDE,F,1960-07-08,Active,Spanish,"6810 Rio PinarNorth Lauderdale, FL, 33068",7547577664,7547577664,"Valdes, Caridad",APT PENDING BY PATIENT,None,Open,Out of network,Diagnostic Radiology,M79.671,Foot Pa and Lat (XRAY) - 73630,Reviewed- Ready to be processed,Pending,External,Adult,Ambetter (Centrum Contract),U9646774901,Exchange,pending auth,"GARCIA, BEATRIZ",Mar 1 2023 3:15PM,../data/raw_referrals/excels/FL REFERRALS 3_1-3_28.xlsx,BEATRIZ,GARCIA,N/A,MARCELLUSANTUDE,AMBETTER FL IFP,None,None,MARCELLUSANTUDE1960-07-08,MARCELLUSANTUDE1960-07-08U964677490116095227882023-03-01Foot Pa and Lat (XRAY) - 73630Diagnostic Radiology,Open,73630,X-RAY EXAM OF FOOT,(754) 475-7766,(754) 475-7766,PAIN IN RIGHT FOOT
5631,"Centrum Medical Holdings, LLC",CH - E Hialeah,"Heliodoro Ruiz, MD",1831529932,2023-03-14,CABRERA,WALKIRIA,F,1962-10-31,Active,Spanish,"3410 Nw 102 Nd St

In [11]:
%%sql

COPY (SELECT DISTINCT "Health Plan" FROM bi_referrals) TO '../data/reference/health_plan.csv' (FORMAT 'csv')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
38
